In [1]:
import tensorflow as tf
import numpy as np
import tfgraphviz as tfg
from tensorboard import notebook
from tensorflow.keras import layers


C:\Users\Mauro\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Mauro\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Mauro\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Mauro\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [3]:
reset_graph()

n_inputs = 28*28 # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [4]:
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X, W) + b
        if activation=="relu":
            return tf.nn.relu(z)
        else:
            return z

In [5]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [6]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

#dense = tf.keras.layers.Dense(32, input_shape=(16,))

with tf.name_scope("dnn"):
    #hidden1 = neuron_layer(X, n_hidden1, name="hidden1", activation="relu")
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1", activation=tf.nn.relu)
    print('Hidden1',hidden1)
    
    #hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation="relu")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2", activation=tf.nn.relu)
    print('Hidden2',hidden2)
    
    #logits = neuron_layer(hidden2, n_outputs, "outputs")
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")    
    print('logits',logits)
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")
    
learning_rate = 0.01

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Hidden1 Tensor("dnn/hidden1/Relu:0", shape=(?, 300), dtype=float32)
Hidden2 Tensor("dnn/hidden2/Relu:0", shape=(?, 100), dtype=float32)
logits Tensor("dnn/outputs/BiasAdd:0", shape=(?, 10), dtype=float32)


In [7]:
n_epochs = 400
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
    save_path = saver.save(sess, "./my_model_finalv1.ckpt")

0 Train accuracy: 0.9 Test accuracy: 0.9024
1 Train accuracy: 0.92 Test accuracy: 0.9254
2 Train accuracy: 0.94 Test accuracy: 0.9372
3 Train accuracy: 0.9 Test accuracy: 0.9416
4 Train accuracy: 0.94 Test accuracy: 0.947
5 Train accuracy: 0.94 Test accuracy: 0.9512
6 Train accuracy: 1.0 Test accuracy: 0.9548
7 Train accuracy: 0.94 Test accuracy: 0.961
8 Train accuracy: 0.96 Test accuracy: 0.9622
9 Train accuracy: 0.94 Test accuracy: 0.9648
10 Train accuracy: 0.92 Test accuracy: 0.9654
11 Train accuracy: 0.98 Test accuracy: 0.9668
12 Train accuracy: 0.98 Test accuracy: 0.968
13 Train accuracy: 0.98 Test accuracy: 0.9702
14 Train accuracy: 1.0 Test accuracy: 0.9696
15 Train accuracy: 0.94 Test accuracy: 0.9718
16 Train accuracy: 0.98 Test accuracy: 0.9726
17 Train accuracy: 1.0 Test accuracy: 0.973
18 Train accuracy: 0.98 Test accuracy: 0.975
19 Train accuracy: 0.96 Test accuracy: 0.9754
20 Train accuracy: 1.0 Test accuracy: 0.9748
21 Train accuracy: 1.0 Test accuracy: 0.9734
22 Train a

181 Train accuracy: 1.0 Test accuracy: 0.9824
182 Train accuracy: 1.0 Test accuracy: 0.9822
183 Train accuracy: 1.0 Test accuracy: 0.9824
184 Train accuracy: 1.0 Test accuracy: 0.9826
185 Train accuracy: 1.0 Test accuracy: 0.9826
186 Train accuracy: 1.0 Test accuracy: 0.9828
187 Train accuracy: 1.0 Test accuracy: 0.9822
188 Train accuracy: 1.0 Test accuracy: 0.9826
189 Train accuracy: 1.0 Test accuracy: 0.9826
190 Train accuracy: 1.0 Test accuracy: 0.9826
191 Train accuracy: 1.0 Test accuracy: 0.9824
192 Train accuracy: 1.0 Test accuracy: 0.9824
193 Train accuracy: 1.0 Test accuracy: 0.9822
194 Train accuracy: 1.0 Test accuracy: 0.9826
195 Train accuracy: 1.0 Test accuracy: 0.9824
196 Train accuracy: 1.0 Test accuracy: 0.9822
197 Train accuracy: 1.0 Test accuracy: 0.9828
198 Train accuracy: 1.0 Test accuracy: 0.9828
199 Train accuracy: 1.0 Test accuracy: 0.9828
200 Train accuracy: 1.0 Test accuracy: 0.9824
201 Train accuracy: 1.0 Test accuracy: 0.9824
202 Train accuracy: 1.0 Test accur

360 Train accuracy: 1.0 Test accuracy: 0.9826
361 Train accuracy: 1.0 Test accuracy: 0.9828
362 Train accuracy: 1.0 Test accuracy: 0.9828
363 Train accuracy: 1.0 Test accuracy: 0.9828
364 Train accuracy: 1.0 Test accuracy: 0.9828
365 Train accuracy: 1.0 Test accuracy: 0.9826
366 Train accuracy: 1.0 Test accuracy: 0.9828
367 Train accuracy: 1.0 Test accuracy: 0.9828
368 Train accuracy: 1.0 Test accuracy: 0.9828
369 Train accuracy: 1.0 Test accuracy: 0.9828
370 Train accuracy: 1.0 Test accuracy: 0.9822
371 Train accuracy: 1.0 Test accuracy: 0.9828
372 Train accuracy: 1.0 Test accuracy: 0.9828
373 Train accuracy: 1.0 Test accuracy: 0.9826
374 Train accuracy: 1.0 Test accuracy: 0.9828
375 Train accuracy: 1.0 Test accuracy: 0.9828
376 Train accuracy: 1.0 Test accuracy: 0.9828
377 Train accuracy: 1.0 Test accuracy: 0.9828
378 Train accuracy: 1.0 Test accuracy: 0.9826
379 Train accuracy: 1.0 Test accuracy: 0.9826
380 Train accuracy: 1.0 Test accuracy: 0.9826
381 Train accuracy: 1.0 Test accur

**Após treinar o modelo, você percebeu alguma diferença na performance do seu modelo? Se sim, qual foi esta diferença?**

R: Após treinar o modelo a diferença na performance do modelo foi mínima

In [8]:
with tf.Session() as sess:
    saver.restore(sess, "./my_model_finalv1.ckpt") # or better, use save_path
    X_new_scaled = X_test[:20]
    Z = logits.eval(feed_dict={X: X_new_scaled})
    y_pred = np.argmax(Z, axis=1)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./my_model_finalv1.ckpt


In [9]:
print("Predicted classes:", y_pred)
print("Actual classes:   ", y_test[:20])

Predicted classes: [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]
Actual classes:    [7 2 1 0 4 1 4 9 5 9 0 6 9 0 1 5 9 7 3 4]


In [10]:
g = tfg.board(tf.get_default_graph())
g.view()

'G.gv.pdf'

In [11]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
#%reload_ext tensorboard

In [12]:
%tensorboard --logdir ./

ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
C:\Users\Mauro\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Mauro\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Mauro\anaconda3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Mau